# Introduction

This section comprises **model training**, **robust analysis**, and **model interaction**

**Model Training** consists of the following steps:

- Tokenization
- Padding
- Train Test split
- Model Training and Results

**Robust Analysis** 

This section has two purposes:

1. Check if the model understands simple concepts of monetary policy (content).

2. Observe the model's prediction in a variety of texts taken from the real world.

**Model Interaction (Only on notebook)**

This section allows the user to input any sentence and observe the model's prediction. 

## GPU Setup

To speed up model training, a GPU needs to be used.

Go to Runtime -> Change Runtime type -> GPU

## SpaCy Setup

In [1]:
"""
PLEASE FOLLOW INSTRUCTIONS ON THIS CELL BEFORE RUNNING THIS NOTEBOOK.
"""

# In order for this file to work, this cell needs to be run first
# Remove the '#' on the last two lines and run the cell.
# When it is done the Runtime needs to be restarted at Runtime -> Restart Runtime
# Please make the last two lines adding '#' at the beginning.
# Please wait until ALL commands have run

#!pip install spacy --upgrade
#!python -m spacy download en_core_web_md

'\nPLEASE FOLLOW INSTRUCTIONS ON THIS CELL BEFORE RUNNING THIS NOTEBOOK.\n'

## Imports

In [2]:
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras import Model
import pandas as pd
import numpy as np
import spacy
import random

In [3]:
# Random seed to guarantee that the results can be reproduced.
tf.keras.utils.set_random_seed(25)

## Getting the repository

In [4]:
# Get files from github repository
!git clone https://github.com/luissejas/central_bank_thesis.git

Cloning into 'central_bank_thesis'...
remote: Enumerating objects: 311, done.
remote: Counting objects: 100% (204/204), done.
remote: Compressing objects: 100% (186/186), done.
remote: Total 311 (delta 15), reused 135 (delta 6), pack-reused 107
Receiving objects: 100% (311/311), 17.08 MiB | 10.95 MiB/s, done.
Resolving deltas: 100% (55/55), done.


# 5.3.3 Tokenization and Padding

In [5]:
# Load the prepared dataset
raw_dataset = pd.read_csv('/content/central_bank_thesis/gru_model/balanced_dataset.csv')
raw_dataset.drop(columns=['Unnamed: 0'], inplace=True)

# Splitting the features (text) and the labels (classification)
raw_text = raw_dataset['text'].values.tolist()
label = raw_dataset['label'].values.tolist()

# Once the text goes through tokenization and padding, its datatype will become an np array
# Therefore, we convert the label from list -> np array to avoid problems when feeding the data into the model
label_as_array = np.asarray(label)

In [6]:
# OPTIONAL
# Visualize the raw text before tokenization
#raw_text[0:3]

In [7]:
# Load the Tokenizer and tokenize each word.
# Note that this will only create a reference for each word as shown in the diagram
# These lines of code will not 'translate' the raw text into tokens

# Create an instance of the tokenizer
tokenizer = Tokenizer(num_words=13000, split=' ')

# Create the references from the raw text
tokenizer.fit_on_texts(raw_text)

# Substitute the raw text into tokens
# Observe the raw text has not been replaced, but a 'translated copy' was created
tokenized_text = tokenizer.texts_to_sequences(raw_text)


In [8]:
# OPTIONAL
# Visualize the raw text after tokenization
#tokenized_text[0:3]

In [9]:
# Pad the sequence
padded_tokenized_text = pad_sequences(tokenized_text, maxlen=72, padding='post')

In [10]:
# OPTIONAL
# Visualize the text after padding
#padded_tokenized_text[0:3]

# 5.4 Model Training

Now that the data is prepared to be fed into the model, other parts need to be setup for the model to begin training.

## 5.4.1 Model Architecture

For this thesis, Tensorflow's Keras API is utilized.

In [11]:
# OPTIONAL
# This cell keeps the original version, in case the user wants to try out different architectures.

#from tensorflow.keras import Model
#from tensorflow.keras.layers import Dense, Embedding, SpatialDropout1D, Input, GRU

#inputs = Input(shape=(72,))
#embedding_layer = Embedding(13000, 150, input_length=72)(inputs)
#dropout_1 = SpatialDropout1D(0.35)(embedding_layer)
#gru_layer = GRU(150)(dropout_1)
#out_layer = Dense(1, activation='sigmoid')(gru_layer)

#model = Model(inputs=inputs, outputs=out_layer, name="gru_classifier")
#model.summary()

In [12]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Embedding, SpatialDropout1D, Input, GRU

# Neural Network Architecture (Functional method)
inputs = Input(shape=(72,))
embedding_layer = Embedding(13000, 150, input_length=72)(inputs)
dropout_1 = SpatialDropout1D(0.35)(embedding_layer)
gru_layer = GRU(150)(dropout_1)
out_layer = Dense(1, activation='sigmoid')(gru_layer)

model = Model(inputs=inputs, outputs=out_layer, name="gru_classifier")
model.summary()

Model: "gru_classifier"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 72)]              0         
                                                                 
 embedding (Embedding)       (None, 72, 150)           1950000   
                                                                 
 spatial_dropout1d (SpatialD  (None, 72, 150)          0         
 ropout1D)                                                       
                                                                 
 gru (GRU)                   (None, 150)               135900    
                                                                 
 dense (Dense)               (None, 1)                 151       
                                                                 
Total params: 2,086,051
Trainable params: 2,086,051
Non-trainable params: 0
__________________________________________

## 5.4.2 Training Parameters

### Model Compilation

In [13]:
# OPTIONAL
# This cell keeps the original version, in case the user wants to try out different architectures.

#from tensorflow.keras.losses import BinaryCrossentropy
#from tensorflow.keras.metrics import BinaryAccuracy
#from tensorflow.keras.optimizers import Adam

# Compiling the model
#loss = BinaryCrossentropy(from_logits=False)
#metric = BinaryAccuracy(name='accuracy')
#optimizer = Adam(learning_rate=0.001)

#model.compile(optimizer=optimizer, loss=loss, metrics=metric)

In [14]:
# Compiling the model
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import BinaryAccuracy
from tensorflow.keras.optimizers import Adam

loss = BinaryCrossentropy(from_logits=False)
metric = BinaryAccuracy(name='accuracy')
optimizer = Adam(learning_rate=0.001)

model.compile(optimizer=optimizer, loss=loss, metrics=metric)

### Model Training

In [15]:
# Train Test Split
# Splitting the data into training and test data
# Random state kept only for the purpose of code reproduction.

x_train, x_test, y_train, y_test = train_test_split(padded_tokenized_text, label_as_array, test_size=0.2, random_state=35)

In [16]:
# Begin model training
history = model.fit(x_train, y_train, epochs=2, batch_size=32, validation_data = (x_test, y_test))

Epoch 1/2
777/777 [==============================] - 16s 11ms/step - loss: 0.5975 - accuracy: 0.6559 - val_loss: 0.4594 - val_accuracy: 0.7934
Epoch 2/2
777/777 [==============================] - 8s 10ms/step - loss: 0.4075 - accuracy: 0.8218 - val_loss: 0.4162 - val_accuracy: 0.8124


# 6.2 Robustness Test

As mentioned earlier, this section evaluates the model on two metrics: content and source. 

It comprised three sections:

0. Functions

  Functions to help the reader focus only on the output and not on the preprocessing.

1. Content

  Some classic terms associated with prosperity and adversity economic periods will be fed into the model and the model's prediction will be analyzed.

2. Source

  Text from different sources but from in sample time periods will be classified by the model. Then the results will be analyzed.

## 6.2.0 Function

In [17]:
# Preparing SpaCy
nlp = spacy.load("en_core_web_md")

# POS list
pos_list = pos_list = ['ADJ', 'ADV', 'VERB', 'NOUN']

# Preparing Vocabulary
vocabulary_as_csv = pd.read_csv('/content/central_bank_thesis/gru_model/vocabulary.csv')
vocabulary = vocabulary_as_csv['word'].values.tolist()

In [18]:
def output_raw_text_model_prediction(raw_text: str, nlp, pos_list: list, vocabulary: list, tokenizer, model):
  """
  Steps:
    1) Filter the words according to the vocabulary

    2) Tokenize and pad raw text

    3) Predict economic period

    4) Output predicted label
  """
  # Step 1)

  raw_text_doc, filtered_words_list = nlp(raw_text), []

  for token in raw_text_doc:
    if token.pos_ in pos_list and token.lemma_ in vocabulary:
      filtered_words_list.append(token.lemma_)
  
  filtered_text = [' '.join(filtered_words_list)]

  # Step 2
  tokenized_text = tokenizer.texts_to_sequences(filtered_text)
  padded_tokenized_text = pad_sequences(tokenized_text, maxlen=72, padding='post')

  # Step 3
  # Activation function is sigmoid function
  # 1 if the result is over 0.5, 0 otherwise
  raw_prediction = (model.predict(padded_tokenized_text) > 0.5).astype('int32')

  # Step 4
  # The prediction output needs to be transformed to get only its value

  model_prediction_unformatted = raw_prediction.tolist()
  model_prediction = [output[0] for output in model_prediction_unformatted]

  if model_prediction[0]:
    prediction_label = "Prosperity"
  else:
    prediction_label = "Adversity"

  return prediction_label

## 6.2.1 Content

In this section, some theoretical knowledge of monetary policy is tested.

### Example 1

In [19]:
text_ = "I believe that high inflation will persist throughout this period."

In [20]:
example_output = output_raw_text_model_prediction(
    raw_text=text_,
    nlp=nlp,
    pos_list=pos_list,
    vocabulary=vocabulary,
    tokenizer = tokenizer,
    model=model
)

example_output

'Adversity'

In [21]:
text_ = "In my view, the current inflation rate is transitory."

In [22]:
example_output = output_raw_text_model_prediction(
    raw_text=text_,
    nlp=nlp,
    pos_list=pos_list,
    vocabulary=vocabulary,
    tokenizer = tokenizer,
    model=model
)

example_output

'Prosperity'

### Example 2

In [23]:
text_ = "Lowering the interest rate to stimulate the economy."

In [24]:
example_output = output_raw_text_model_prediction(
    raw_text=text_,
    nlp=nlp,
    pos_list=pos_list,
    vocabulary=vocabulary,
    tokenizer = tokenizer,
    model=model
)

example_output

'Prosperity'

In [25]:
text_ = "We raise the interest rate to cool down prices."

In [26]:
example_output = output_raw_text_model_prediction(
    raw_text=text_,
    nlp=nlp,
    pos_list=pos_list,
    vocabulary=vocabulary,
    tokenizer = tokenizer,
    model=model
)

example_output

'Adversity'

### 6.2.2 Real World Examples

In this section, I use the model to classify real world text from different sources.

The idea is to test whether the model understand announcements about the economy in simple language.

Additionally, I test whether the model works on different styles of speech.

### Example 1

Janet Yellen interview with CNBC in 2017

Source: https://www.cnbc.com/2017/12/13/yellens-only-regret-as-fed-chair-low-inflation.html

In [27]:
# Normal Language
text_ = "I feel good about the economy outlook, I feel good that the labor market is in a very much stronger place than it was eight years ago. We’ve created 17 million jobs, we’ve got a good, strong labor market and a very low unemployment rate. That’s been tremendously important to the well-being of American households and workers."

In [28]:
example_output = output_raw_text_model_prediction(
    raw_text=text_,
    nlp=nlp,
    pos_list=pos_list,
    vocabulary=vocabulary,
    tokenizer = tokenizer,
    model=model
)

example_output

'Prosperity'

In [29]:
# Technical Language
text_ = "We have a 2 percent symmetric inflation objective. For a number of years now, inflation has been running under 2 percent, and I consider it an important priority to make sure that inflation doesn’t chronically undershoot our 2 percent objective."

In [30]:
example_output = output_raw_text_model_prediction(
    raw_text=text_,
    nlp=nlp,
    pos_list=pos_list,
    vocabulary=vocabulary,
    tokenizer = tokenizer,
    model=model
)

example_output

'Prosperity'

### Example 2

Excerpt from Mario Draghi's (European Central Bank) speech

Source: https://www.ecb.europa.eu/press/key/date/2012/html/sp120726.en.html

In [31]:
# Talk during Euro crisis
text_ = "The progress in undertaking deficit control, structural reforms has been remarkable. And they will have to continue to do so. But the pace has been set and all the signals that we get is that they don’t relent, stop reforming themselves. It’s a complex process because for many years, very little was done – I will come to this in a moment."

In [32]:
example_output = output_raw_text_model_prediction(
    raw_text=text_,
    nlp=nlp,
    pos_list=pos_list,
    vocabulary=vocabulary,
    tokenizer = tokenizer,
    model=model
)

example_output

'Prosperity'

### Example 3

FOMC Statement 28 June 2000

Source: https://fraser.stlouisfed.org/title/federal-open-market-committee-meeting-minutes-transcripts-documents-677/meeting-june-27-28-2000-23199/content/pdf/20000628statement

In [33]:
text_ = "The Federal Open Market Committee at its meeting today decided to maintain the existing stance of monetary policy, keeping its target for the federal funds rate at 6-1/2 percent. Recent data suggest that the expansion of aggregate demand may be moderating toward a pace closer to the rate of growth of the economy's potential to produce. Although core measures of prices are rising slightly faster than a year ago, continuing rapid advances in productivity have been containing costs and holding down underlying price pressures. Nonetheless, signs that growth in demand is moving to a sustainable pace are still tentative and preliminary, and the utilization of the pool of available workers remains at an unusually high level. In these circumstances, and against the background of its long-term goals of price stability and sustainable economic growth and of the information currently available, the Committee believes the risks continue to be weighted mainly toward conditions that may generate heightened inflation pressures in the foreseeable future."

In [34]:
example_output = output_raw_text_model_prediction(
    raw_text=text_,
    nlp=nlp,
    pos_list=pos_list,
    vocabulary=vocabulary,
    tokenizer = tokenizer,
    model=model
)

example_output

'Adversity'

## 6.2.3 Try it Yourself

In this section the user can input any text and see the model's prediction.

In [35]:
text_ = "INSERT YOUR SENTENCE HERE"

In [ ]:
example_output = output_raw_text_model_prediction(
    raw_text=text_,
    nlp=nlp,
    pos_list=pos_list,
    vocabulary=vocabulary,
    tokenizer = tokenizer,
    model=model
)

example_output